# Importa los módulos necesarios y establece configuraciones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import funs
import seaborn as sns
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler
init_notebook_mode(connected=True)
pd.options.display.max_rows = 10

## Importa el .csv que contiene las mediciones de todos los nodos

In [2]:
df = pd.read_csv('all_nodos.csv')
df

,id_Variable,id_nodo,Temperatura,Humedad,Hum. Tierra,Nivel_UV,INT. LUMÍN,Carga,RSSI,T_SERVIDOR,T_NODO,TRY,FECHA,AÑO,MES,DIA,HORA,MINUTO,TIPO_Nodo
0,1,4,0,0,0.00,0,0,0.00,39,336,170,63,NaN,2018,4,27,8,17,3
1,2,5,0,0,0.00,0,0,0.00,0,1926,1678,63,NaN,2018,4,27,8,17,3
2,3,3,30,42,0.01,0,1038,0.60,55,0,117,68,NaN,2018,4,27,8,17,0
3,4,2,30,44,0.01,0,791,-11.67,56,0,105,0,NaN,2018,4,27,8,18,0
4,5,6,30,44,0.01,12,5777,0.59,46,291,125,0,NaN,2018,4,27,8,18,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50496,50497,8,60,78,0.98,0,5853,1.33,87,389,139,0,NaN,2018,6,11,9,17,3
50497,50498,3,32,29,1.14,0,5835,-11.67,84,0,0,0,NaN,2018,6,11,9,17,0
50498,50499,2,32,17,0.82,0,5853,1.69,83,0,0,0,NaN,2018,6,11,9,17,0
50499,50500,5,33,29,6.43,0,5865,1.50,82,0,0,0,NaN,2018,6,11,9,17,0


## Suprime las columnas que no se necesitan

In [3]:
try:
    df.drop(columns=['T_NODO', 'TRY', 'FECHA', 'Carga', 'RSSI', 'T_SERVIDOR', 'TIPO_Nodo'], inplace = True)
except KeyError:
    pass
df

,id_Variable,id_nodo,Temperatura,Humedad,Hum. Tierra,Nivel_UV,INT. LUMÍN,AÑO,MES,DIA,HORA,MINUTO
0,1,4,0,0,0.00,0,0,2018,4,27,8,17
1,2,5,0,0,0.00,0,0,2018,4,27,8,17
2,3,3,30,42,0.01,0,1038,2018,4,27,8,17
3,4,2,30,44,0.01,0,791,2018,4,27,8,18
4,5,6,30,44,0.01,12,5777,2018,4,27,8,18
...,...,...,...,...,...,...,...,...,...,...,...,...
50496,50497,8,60,78,0.98,0,5853,2018,6,11,9,17
50497,50498,3,32,29,1.14,0,5835,2018,6,11,9,17
50498,50499,2,32,17,0.82,0,5853,2018,6,11,9,17
50499,50500,5,33,29,6.43,0,5865,2018,6,11,9,17


## Atrasa la columna HORA del mes mayo en tres unidades, la hora estaba adelantada para esos registros

In [4]:
ind_rem = [[x,int(df.HORA[x])-3] for x in range(len(df)) if df.MES[x] < 6]
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'HORA'] = pd.DataFrame(ind_rem).iloc[:,1].values
ind_rem = [[x, df.HORA[x]+24,df.DIA[x]-1] for x in range(len(df)) if df.HORA[x]<0]
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'HORA'] = pd.DataFrame(ind_rem).iloc[:,1].values
df.loc[pd.DataFrame(ind_rem).iloc[:,0],'DIA'] = pd.DataFrame(ind_rem).iloc[:,2].values

## Cambia los nombres de las columnas

In [5]:
old_names = ['id_Variable','id_nodo','Temperatura','Humedad','Hum. Tierra','Nivel_UV','INT. LUMÍN'] 
new_names = ['id_Variable','NODO','TEMPERATURA','HUMEDAD RELATIVA','HUMEDAD DE LA TIERRA','NIVEL UV','INTENSIDAD LUMÍNICA']
df.rename(columns=dict(zip(old_names, new_names)), inplace=True)
df

,id_Variable,NODO,TEMPERATURA,HUMEDAD RELATIVA,HUMEDAD DE LA TIERRA,NIVEL UV,INTENSIDAD LUMÍNICA,AÑO,MES,DIA,HORA,MINUTO
0,1,4,0,0,0.00,0,0,2018,4,27,5,17
1,2,5,0,0,0.00,0,0,2018,4,27,5,17
2,3,3,30,42,0.01,0,1038,2018,4,27,5,17
3,4,2,30,44,0.01,0,791,2018,4,27,5,18
4,5,6,30,44,0.01,12,5777,2018,4,27,5,18
...,...,...,...,...,...,...,...,...,...,...,...,...
50496,50497,8,60,78,0.98,0,5853,2018,6,11,9,17
50497,50498,3,32,29,1.14,0,5835,2018,6,11,9,17
50498,50499,2,32,17,0.82,0,5853,2018,6,11,9,17
50499,50500,5,33,29,6.43,0,5865,2018,6,11,9,17


## Se introduce un cero antes de las unidades de las columnas MES, DIA, HORA, MINUTO

In [6]:
ind_rem = {}
for i in ['MES','DIA','HORA','MINUTO']:
    ind_rem[i] = [[x,'0'+str(df[i][x])] for x in range(len(df)) if df[i][x] < 10]
    df.loc[pd.DataFrame(ind_rem[i]).iloc[:,0],i] = pd.DataFrame(ind_rem[i]).iloc[:,1].values
df

,id_Variable,NODO,TEMPERATURA,HUMEDAD RELATIVA,HUMEDAD DE LA TIERRA,NIVEL UV,INTENSIDAD LUMÍNICA,AÑO,MES,DIA,HORA,MINUTO
0,1,4,0,0,0.00,0,0,2018,04,27,05,17
1,2,5,0,0,0.00,0,0,2018,04,27,05,17
2,3,3,30,42,0.01,0,1038,2018,04,27,05,17
3,4,2,30,44,0.01,0,791,2018,04,27,05,18
4,5,6,30,44,0.01,12,5777,2018,04,27,05,18
...,...,...,...,...,...,...,...,...,...,...,...,...
50496,50497,8,60,78,0.98,0,5853,2018,06,11,09,17
50497,50498,3,32,29,1.14,0,5835,2018,06,11,09,17
50498,50499,2,32,17,0.82,0,5853,2018,06,11,09,17
50499,50500,5,33,29,6.43,0,5865,2018,06,11,09,17


## Se crea una columna con la estrucutura AÑO+MES+DIA+HORA+MINUTO

In [7]:
df['TIME'] = [str(df.AÑO[i])+str(df.MES[i])+str(df.DIA[i])+str(df.HORA[i])+str(df.MINUTO[i]) for i in range(len(df))]
df.TIME

0        201804270517
1        201804270517
2        201804270517
3        201804270518
4        201804270518
             ...     
50496    201806110917
50497    201806110917
50498    201806110917
50499    201806110917
50500    201806110918
Name: TIME, Length: 50501, dtype: object

## Convierte la columna TIME al fomato Timestamp

In [8]:
df['TIME'] = [pd.Timestamp(x) for x in df['TIME']]
df

,id_Variable,NODO,TEMPERATURA,HUMEDAD RELATIVA,HUMEDAD DE LA TIERRA,NIVEL UV,INTENSIDAD LUMÍNICA,AÑO,MES,DIA,HORA,MINUTO,TIME
0,1,4,0,0,0.00,0,0,2018,04,27,05,17,2018-04-27 05:17:00
1,2,5,0,0,0.00,0,0,2018,04,27,05,17,2018-04-27 05:17:00
2,3,3,30,42,0.01,0,1038,2018,04,27,05,17,2018-04-27 05:17:00
3,4,2,30,44,0.01,0,791,2018,04,27,05,18,2018-04-27 05:18:00
4,5,6,30,44,0.01,12,5777,2018,04,27,05,18,2018-04-27 05:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50496,50497,8,60,78,0.98,0,5853,2018,06,11,09,17,2018-06-11 09:17:00
50497,50498,3,32,29,1.14,0,5835,2018,06,11,09,17,2018-06-11 09:17:00
50498,50499,2,32,17,0.82,0,5853,2018,06,11,09,17,2018-06-11 09:17:00
50499,50500,5,33,29,6.43,0,5865,2018,06,11,09,17,2018-06-11 09:17:00


## Suprime las columnas que no se van a utilizar

In [9]:
try:
    df.drop(columns = ['id_Variable','AÑO','MES','DIA','MINUTO'], inplace = True)
except KeyError:
    pass

## Elimina los elementos de df anteriores al registro 2882, correspondientes a pruebas

In [10]:
if len(df) == 50501:
    df = df.iloc[2822::,:].reset_index(drop = True)

In [11]:
df.index = df.TIME
df.drop(columns = 'TIME', inplace = True)
df

,NODO,TEMPERATURA,HUMEDAD RELATIVA,HUMEDAD DE LA TIERRA,NIVEL UV,INTENSIDAD LUMÍNICA,HORA
TIME,,,,,,,
2018-05-09 09:38:00,2,27,19,74.97,0,5853,09
2018-05-09 09:38:00,6,0,0,0.00,0,0,09
2018-05-09 09:49:00,3,27,65,94.07,0,5824,09
2018-05-09 09:49:00,6,0,0,0.00,0,0,09
2018-05-09 09:49:00,2,28,19,72.72,0,5865,09
...,...,...,...,...,...,...,...
2018-06-11 09:17:00,8,60,78,0.98,0,5853,09
2018-06-11 09:17:00,3,32,29,1.14,0,5835,09
2018-06-11 09:17:00,2,32,17,0.82,0,5853,09


In [12]:
# convierte a entero los registros de la columna HORA
df.HORA = [int(df.HORA[x]) for x in range(len(df))]

In [13]:
n = df.NODO.value_counts()
# imprime los nodos contenidos en df y la cantidad de registros que posee
print(n)
nodos  = n.index.tolist()

3    14363
5    11771
2    11042
8     8440
6     1669
4      394
Name: NODO, dtype: int64


## Separa los nodos

In [14]:
tablas = {}
for i in nodos:
    tablas[i] = (df.query('NODO == %s' %i)).copy()#.reset_index(drop=False))
    tablas[i].drop(columns = ['NODO'],inplace=True)
    tablas[i] = tablas[i][~tablas[i].index.duplicated(keep='first')]

### Guarda los registros de los nodos en datasets separados


In [15]:
for i in nodos:
    (tablas[i]).to_csv('medidas_nodo%d.csv'%i)

### Guarda un nuevo .csv de las mediciones

In [16]:
print(len(df))
df = df[~df.index.duplicated(keep='first')]
print(len(df))
df.to_csv('medidas_update.csv')

47679
16215
